In [1]:
import pandas as pd
import os
from pathlib import Path
import toolkitETL
import importlib
importlib.reload(toolkitETL)



<module 'toolkitETL' from 'c:\\Users\\pedro\\OneDrive\\Área de Trabalho\\Pedro\\Portifolio\\ETL-ML-Dengue\\src\\etl\\toolkitETL.py'>

In [2]:
## Estabelecendo diretórios base
diretorio_src_etl = os.getcwd()
diretorio_src = Path(diretorio_src_etl).parent.absolute()
diretorio_base = Path(diretorio_src).parent.absolute()
diretorio_base = "D:\\Mestrado_pedro"
diretorio_data = os.path.join(diretorio_base, 'data')

# Bronze
diretorio_bronze = os.path.join(diretorio_data, 'bronze')
diretorio_ibge_pib = os.path.join(diretorio_bronze, 'ibge_pib')
diretorio_ibge_pib_tsv = os.path.join(diretorio_ibge_pib, 'tsv')

# Silver
diretorio_silver = os.path.join(diretorio_data, 'silver')
diretorio_ibge_pib_silver = os.path.join(diretorio_silver, 'ibge_pib')
diretorio_ibge_pib_tsv_silver = os.path.join(diretorio_ibge_pib_silver, 'tsv')


## Criação de pastas para armazenar os arquivos
os.makedirs(os.path.join(diretorio_data), exist_ok=True)

# Bronze
os.makedirs(os.path.join(diretorio_bronze), exist_ok=True)
os.makedirs(os.path.join(diretorio_ibge_pib), exist_ok=True)
os.makedirs(os.path.join(diretorio_ibge_pib_tsv), exist_ok=True)

# Silver
os.makedirs(os.path.join(diretorio_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_ibge_pib_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_ibge_pib_tsv_silver), exist_ok=True)



In [3]:
arquivos_tsv = [arquivo for arquivo in os.listdir(diretorio_ibge_pib_tsv) if arquivo.endswith("tsv")]
arquivos_tsv

['pib dos municípios - base de dados 2010-2021.tsv']

In [48]:
column_map = {
    'Ano': 'Ano_Notificacao',
    'Código da Grande Região': 'Codigo_Grande_Regiao',
    'Nome da Grande Região': 'Nome_Grande_Regiao',
    'Código da Unidade da Federação': 'Codigo_UF',
    'Sigla da Unidade da Federação': 'UF',
    'Nome da Unidade da Federação': 'Nome_UF',
    'Código do Município': 'Codigo_Municipio_completo',
    'Nome do Município': 'Municipio',
    'Região Metropolitana': 'Regiao_Metropolitana',
    'Código da Mesorregião': 'Codigo_Mesorregiao',
    'Nome da Mesorregião': 'Nome_Mesorregiao',
    'Código da Microrregião': 'Codigo_Microrregiao',
    'Nome da Microrregião': 'Nome_Microrregiao',
    'Código da Região Geográfica Imediata': 'Codigo_Regiao_Imediata',
    'Nome da Região Geográfica Imediata': 'Nome_Regiao_Imediata',
    'Município da Região Geográfica Imediata': 'Municipio_Regiao_Imediata',
    'Código da Região Geográfica Intermediária': 'Codigo_Regiao_Intermediaria',
    'Nome da Região Geográfica Intermediária': 'Nome_Regiao_Intermediaria',
    'Município da Região Geográfica Intermediária': 'Municipio_Regiao_Intermediaria',
    'Código Concentração Urbana': 'Codigo_Concentracao_Urbana',
    'Nome Concentração Urbana': 'Nome_Concentracao_Urbana',
    'Tipo Concentração Urbana': 'Tipo_Concentracao_Urbana',
    'Código Arranjo Populacional': 'Codigo_Arranjo_Populacional',
    'Nome Arranjo Populacional': 'Nome_Arranjo_Populacional',
    'Hierarquia Urbana': 'Hierarquia_Urbana',
    'Hierarquia Urbana (principais categorias)': 'Hierarquia_Urbana_Principais_Categorias',
    'Código da Região Rural': 'Codigo_Regiao_Rural',
    'Nome da Região Rural': 'Nome_Regiao_Rural',
    'Região rural (segundo classificação do núcleo)': 'Regiao_Rural_Classificacao_Nucleo',
    'Amazônia Legal': 'Amazonia_Legal',
    'Semiárido': 'Semiarido',
    'Cidade-Região de São Paulo': 'Cidade_Regiao_de_Sao_Paulo',
    'Valor adicionado bruto da Agropecuária, \na preços correntes\n(R$ 1.000)': 'VAB_Agropecuaria',
    'Valor adicionado bruto da Indústria,\na preços correntes\n(R$ 1.000)': 'VAB_Industria',
    'Valor adicionado bruto dos Serviços,\na preços correntes \n- exceto Administração, defesa, educação e saúde públicas e seguridade social\n(R$ 1.000)': 'VAB_Servicos',
    'Valor adicionado bruto da Administração, defesa, educação e saúde públicas e seguridade social, \na preços correntes\n(R$ 1.000)': 'VAB_Adm_Publica',
    'Valor adicionado bruto total, \na preços correntes\n(R$ 1.000)': 'VAB_Total',
    'Impostos, líquidos de subsídios, sobre produtos, \na preços correntes\n(R$ 1.000)': 'Impostos',
    'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)': 'PIB',
    'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)': 'PIB_Per_Capita',
    'Atividade com maior valor adicionado bruto': 'Atividade_Maior_VAB',
    'Atividade com segundo maior valor adicionado bruto': 'Atividade_Seg_Maior_VAB',
    'Atividade com terceiro maior valor adicionado bruto': 'Atividade_Ter_Maior_VAB'
}

columns_to_convert = [
    "codigo_municipio_completo", 'codigo_arranjo_populacional', 'codigo_concentracao_urbana',
    'codigo_municipio', 'codigo_regiao_rural', 'codigo_regiao_intermediaria', 'codigo_regiao_imediata',
    'codigo_microrregiao', 'codigo_mesorregiao', 'codigo_uf', 'ano_notificacao']

In [52]:
for arquivo in arquivos_tsv:
    df = pd.read_csv(os.path.join(diretorio_ibge_pib_tsv, arquivo), sep='\t')
    df = df.rename(columns=column_map)
    df['Codigo_Municipio'] = df['Codigo_Municipio_completo'].apply(lambda x: str(x)[:-1])
    # Itera sobre cada coluna e tenta converter para inteiro
    df.columns = df.columns.str.lower()
    for column in columns_to_convert:
        df[column] = pd.to_numeric(df[column], errors='coerce').astype('Int64')
    df.to_csv(os.path.join(diretorio_ibge_pib_tsv_silver, arquivo), sep='\t', index=False)

In [5]:
diretorio_dotenv = os.path.join(diretorio_base, ".env")

cursor, conn = toolkitETL.conectar_banco(diretorio_dotenv)
for arquivo_tsv in arquivos_tsv:
    
    caminho_arquivo = os.path.join(diretorio_ibge_pib_tsv_silver, arquivo_tsv)
    toolkitETL.inserir_no_banco(cursor, conn, caminho_arquivo, 'ibge_pib', delimiter='\t', diretorio_dotenv=diretorio_dotenv)
